In [1]:
from enum import Enum
from sqlalchemy import Column, Integer, String, create_engine, DateTime, Boolean, ForeignKey, select
from sqlalchemy.orm import declarative_base, mapped_column, Mapped, relationship, sessionmaker
from sqlalchemy.exc import SQLAlchemyError
from typing import Optional, Dict, Any, List
from datetime import datetime

DATABASE_URL = "mysql+pymysql://root:@127.0.0.1:3306/paruvendu"
engine = create_engine(DATABASE_URL, echo=True)

Base = declarative_base()

class AnnouncementType(Enum):
    MAISON = 1
    APPARTEMENT = 2

class AnnouncerType(Enum):
    PARTICULIER = 1
    PROFFESSIONELLE = 2
    NB = 2

class Agency(Base):
    __tablename__ = 'announcer'
    id = Column(Integer, primary_key=True)
    name = Column(String(100), nullable=True)
    address = Column(String(100), nullable=True)
    slogan = Column(String(200), nullable=True)
    description = Column(String(1000), nullable=True)
    telephone = Column(String(15), nullable=True)
    yearMember = Column(String(15), nullable=True)
    announcements: Mapped[List["Announcement"]] = relationship(
        "Announcement",
        back_populates="agency",
        cascade="all, delete-orphan",
        lazy="select"
    )

class Announcement(Base):
    __tablename__ = 'announcements'
    id = Column(Integer, primary_key=True)
    ref = Column(String(20), nullable=True)
    title = Column(String(50), nullable=True)
    description = Column(String(1000), nullable=True)
    price = Column(Integer, nullable=True)
    price_per_meter = Column(Integer, nullable=True)
    publish_at = Column(DateTime, nullable=True)
    exclusive = Column(Boolean, nullable=True)
    updated_at = Column(DateTime, nullable=True)
    dpe = Column(String(1), nullable=True, info={'comment': 'DPE rating A to G'})
    url = Column(String(200), nullable=True)
    type = Column(Integer, nullable=True)
    agency_id = mapped_column(ForeignKey("announcer.id"))
    agency: Mapped[Agency] = relationship(
        "Agency",
        back_populates="announcements",
        lazy="joined"
    )

class Caracteristic(Base):
    __tablename__ = 'estate'
    id = Column(Integer, primary_key=True)
    announcement_id = mapped_column(ForeignKey("announcements.id"))
    announcement: Mapped[Announcement] = relationship()
    nb_rooms = Column(Integer, nullable=True)
    nb_bedrooms = Column(Integer, nullable=True)
    location = Column(String(50), nullable=True, info={'comment': 'Location'})
    parking_garage = Column(Boolean, nullable=True)
    garden = Column(Boolean, nullable=True)
    balcony_terrace = Column(Boolean, nullable=True)
    annexes = Column(String(10), nullable=True)
    access = Column(String(50), nullable=True)
    arrangement = Column(String(50), nullable=True)
    dependence = Column(String(50), nullable=True)
    outside = Column(String(50), nullable=True)
    connectivity_index = Column(Integer, nullable=True)
    fiber_eligibility_rate = Column(Integer, nullable=True)
    general_information = Column(String(200), nullable=True)
    surface = Column(String(200), nullable=True)
    agencement = Column(String(200), nullable=True)

def init_db():
    Base.metadata.create_all(engine)

def drop_db():
    Base.metadata.drop_all(engine)

def reset_db():
    drop_db()
    init_db()
    print("Database reset completed.")

# Initialiser la base de données (créer les tables)
reset_db()


2025-11-26 16:58:53,414 INFO sqlalchemy.engine.Engine SELECT DATABASE()
2025-11-26 16:58:53,416 INFO sqlalchemy.engine.Engine [raw sql] {}
2025-11-26 16:58:53,418 INFO sqlalchemy.engine.Engine SELECT @@sql_mode
2025-11-26 16:58:53,419 INFO sqlalchemy.engine.Engine [raw sql] {}
2025-11-26 16:58:53,421 INFO sqlalchemy.engine.Engine SELECT @@lower_case_table_names
2025-11-26 16:58:53,422 INFO sqlalchemy.engine.Engine [raw sql] {}
2025-11-26 16:58:53,424 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-11-26 16:58:53,424 INFO sqlalchemy.engine.Engine DESCRIBE `paruvendu`.`announcer`
2025-11-26 16:58:53,426 INFO sqlalchemy.engine.Engine [raw sql] {}
2025-11-26 16:58:53,458 INFO sqlalchemy.engine.Engine DESCRIBE `paruvendu`.`announcements`
2025-11-26 16:58:53,459 INFO sqlalchemy.engine.Engine [raw sql] {}
2025-11-26 16:58:53,469 INFO sqlalchemy.engine.Engine DESCRIBE `paruvendu`.`estate`
2025-11-26 16:58:53,470 INFO sqlalchemy.engine.Engine [raw sql] {}
2025-11-26 16:58:53,480 INFO sqlalc

In [2]:
SessionLocal = sessionmaker(bind=engine, autoflush=False, autocommit=False)

def create_announcement(
    session,
    announcement_data: Dict[str, Any],
    announcer_data: Dict[str, Any],
    estate_data: Optional[Dict[str, Any]] = None,
    *,
    dedupe_agency_by: Optional[list] = None
):
    """
    Crée une annonce, associe une agence (sans la dupliquer) et crée les caractéristiques optionnelles.

    - session: SQLAlchemy session (ex: SessionLocal()).
    - announcement_data: dict contenant les champs de Announcement (ref, title, price, publish_at, url, type, etc.)
    - announcer_data: dict avec les champs de Agency (name, telephone, address, slogan?, description?)
    - estate_data: dict optionnel pour Caracteristic.
    - dedupe_agency_by: liste des champs sur lesquels dédupliquer l'agence (par défaut ['name', 'telephone']).

    Retourne l'objet Announcement (attaché à la session) après commit.
    """
    if dedupe_agency_by is None:
        dedupe_agency_by = ["name"]

    try:
        # Transaction
        with session.begin():
            # 1) chercher l'agence existante en se basant sur les champs fournis
            agency_filters = {}
            for key in dedupe_agency_by:
                if key in announcer_data and announcer_data[key] is not None:
                    agency_filters[key] = announcer_data[key]

            agency = None
            if agency_filters:
                stmt = select(Agency).filter_by(**agency_filters)
                agency = session.execute(stmt).scalars().first()

            # 2) si pas trouvée, créer l'agence (en utilisant seulement les champs valides)
            if agency is None:
                # Construire un dict contenant uniquement les colonnes existantes sur Agency
                allowed_agency_fields = {
                    "name", "address", "slogan", "description", "telephone"
                }
                agency_kwargs = {k: announcer_data.get(k) for k in allowed_agency_fields if k in announcer_data}
                agency = Agency(**agency_kwargs)
                session.add(agency)
                # flush pour obtenir agency.id si nécessaire plus loin
                session.flush()

            # 3) vérifier si l'annonce existe déjà (optionnel) -> éviter duplication si ref unique
            announcement = None
            ref = announcement_data.get("ref")
            if ref:
                stmt = select(Announcement).filter_by(ref=ref, agency_id=agency.id)
                announcement = session.execute(stmt).scalars().first()

            # 4) créer ou mettre à jour l'annonce
            allowed_announcement_fields = {
                "ref", "title", "description", "price", "price_per_meter", "publish_at", "exclusive",
                "updated_at", "dpe", "url", "type"
            }
            ann_kwargs = {k: announcement_data.get(k) for k in allowed_announcement_fields if k in announcement_data}

            # si l'utilisateur passe un Enum AnnouncementType, convertir en int
            if "type" in ann_kwargs and ann_kwargs["type"] is not None:
                t = ann_kwargs["type"]
                # supporte int, Enum, ou string représentant un int
                if hasattr(t, "value"):
                    ann_kwargs["type"] = int(t.value)
                else:
                    try:
                        ann_kwargs["type"] = int(t)
                    except Exception:
                        # laisse tel quel (DB attend un int mais on assume que l'appelant envoie correct)
                        pass

            if announcement is None:
                announcement = Announcement(**ann_kwargs)
                # fixer la relation vers l'agence
                announcement.agency = agency
                session.add(announcement)
                session.flush()  # pour obtenir announcement.id si besoin
            else:
                # si déjà existante, on met à jour les champs fournis
                for k, v in ann_kwargs.items():
                    setattr(announcement, k, v)

            # 5) créer les caractéristiques si fournies
            if estate_data is not None:
                # vérifier s'il existe déjà une ligne de caractéristique pour cette annonce
                stmt = select(Caracteristic).filter_by(announcement_id=announcement.id)
                existing_car = session.execute(stmt).scalars().first()
                allowed_car_fields = {
                    "parking_garage", "garden","nb_rooms", "nb_bedrooms", "balcony_terrace", "annexes", "access",
                    "arrangement", "dependence", "outside", "connectivity_index","location",
                    "fiber_eligibility_rate", "general_information", "agencement", "surface"
                }
                car_kwargs = {k: estate_data.get(k) for k in allowed_car_fields if k in estate_data}

                if existing_car is None:
                    car_kwargs["announcement_id"] = announcement.id
                    caracteristic = Caracteristic(**car_kwargs)
                    session.add(caracteristic)
                else:
                    for k, v in car_kwargs.items():
                        setattr(existing_car, k, v)

            # commit fait automatiquement par session.begin() context manager
            # retourner l'annonce (attachée au session)
            return announcement

    except SQLAlchemyError as exc:
        # rollback implicite si exception dans session.begin()
        # relancer après log ou gestion si nécessaire
        raise


# --- Exemple d'utilisation ---
# if __name__ == "__main__":
#     s = SessionLocal()

#     agency_payload = {
#         "name": "Agence Dupont",
#         "address": "1 rue de Paris, 75001 Paris",
#         "telephone": "0123456789",
#         "yearMember": "2020",
#         "description": "Agence spécialisée dans les maisons familiales"
#     }

#     announcement_payload = {
#         "ref": "PV-2025-0001",
#         "title": "Maison familiale 4 pièces",
#         "description": "Belle maison avec jardin, proche écoles et commerces.",
    #     "price": 350000,
    #     "price_per_meter": 2500,
    #     "publish_at": datetime.now(),
    #     "exclusive": True,
    #     "updated_at": datetime.now(),
    #     "dpe": "B",
    #     "url": "https://paruvendu.example/annonce/1",
    #     "type": AnnouncementType.MAISON.value,  # int pour la DB
    # }

    # caracteristic_payload = {
    #     "nb_rooms": 5,
    #     "nb_bedrooms": 3,
    #     "location": "Paris 8ème",
    #     "parking_garage": True,
    #     "garden": False,
    #     "balcony_terrace": True,
    #     "annexes": "Cave",
    #     "access": "Digicode, Interphone",
    #     "arrangement": "Traversant",
    #     "outside": "Cour intérieure",
    #     "connectivity_index": 95,
    #     "fiber_eligibility_rate": 100,
    #     "general_information": "Proche métro et commerces"
    # }

    # ann = create_announcement(
    #     s,
    #     announcement_data=announcement_payload,
    #     announcer_data=agency_payload,
    #     estate_data=caracteristic_payload
    # )

    # print("Created announcement id:", ann.id)
    # s.close()


In [3]:
import pandas as pd
import ast
paruvendu_data = pd.read_csv("results.csv",  header=None)
paruvendu_data[1] = paruvendu_data[1].apply(ast.literal_eval)
df_dico = pd.json_normalize(paruvendu_data[1])
df_dico

,title,location,descriptionTitle,description,nbp,surf,prix,enseigneInfosvendeur,agencement,general,annexe,dependance,publishedAt,reference
0,Vente Appartement 1 pièce 28 m²,Paris 13 (75013),Description T1 à Paris 13,Appartement 75013 - PARIS - BUTTE AUX CAILLES ...,1,28,240 750,EFFICITY Sukhanakova Elena,NaN,NaN,NaN,NaN,NaN,NaN
1,Vente Appartement 1 pièce 28 m²,Aubervilliers (93300),Description T1 à Aubervilliers,Appartement 1 pièce Aubervilliers 27 m2 A vend...,1,28,135 000,L'ADRESSE AUBERVILLIERS,NaN,NaN,NaN,NaN,NaN,NaN
2,Vente Appartement 2 pièces 42 m²,Saint denis (93200),Description T2 à Saint denis,Appartement Pierrefitte Sur Seine 2 pièces 42 ...,2,42,137 800,L'ADRESSE AUBERVILLIERS,NaN,NaN,NaN,NaN,NaN,NaN
3,Vente Appartement 1 pièce 24 m²,Carrieres sous poissy (78955),Description T1 à Carrieres sous poissy,APPARTEMENT POISSY - 1 pièce(s) - 24.46m2 IDÉA...,1,24,123 000,HAG IMMOBILIER,NaN,NaN,NaN,NaN,NaN,NaN
4,Vente Appartement 2 pièces 52 m²,La garenne-colombes (92250),Description T2 à La garenne-colombes,Vente Appartemnt Au c?ur des vallées à proximi...,2,52,435 000,MAXIHOME,NaN,NaN,NaN,NaN,NaN,NaN
5,Vente Appartement 1 pièce 30 m²,Carrieres sous poissy (78955),Description T1 à Carrieres sous poissy,studio vendu loue Dans un quartier calme de Ca...,1,30,130 000,HAG IMMOBILIER,NaN,NaN,NaN,NaN,NaN,NaN
6,Vente Maison 6 pièces 160 m²,Adainville (78113),Description T6 à Adainville,MAISON BEAU JARDIN PISCINE COUVERTE ET SOUS-SO...,6,160,415 000,STEPHAN CEREZO IMMOBILIER,NaN,NaN,NaN,NaN,NaN,NaN
7,Vente Maison 8 pièces 190 m²,Poigny la foret (78125),Description T8 à Poigny la foret,Maison Poigny La Foret 8 pièce(s) 190 m2 Dans ...,8,190,598 000,STEPHAN CEREZO IMMOBILIER,NaN,NaN,NaN,NaN,NaN,NaN
8,,Bennecourt (78270),,EXCLUSIVITE - Propriété pleine de charme avec ...,8,208,None,,NaN,NaN,NaN,NaN,NaN,NaN
9,Vente Maison 5 pièces 163 m²,Bourdonne (78113),Description T5 à Bourdonne,Maison 5 pièce(s) 163 m2 Dans village entre Ga...,5,163,400 000,STEPHAN CEREZO IMMOBILIER,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
import math

def clean_nan(value):
    if isinstance(value, float) and math.isnan(value):
        return "NB"   # valeur par défaut quand c'est NaN
    return value

In [5]:

announcement_payloads = []
agency_payload = []
caracteristic_payload = []

for i in range(len(df_dico)):
    payload = {
        "url": clean_nan(paruvendu_data[0].iloc[i]),
        "ref": clean_nan(df_dico["reference"].iloc[i]),
        "title": clean_nan(df_dico["title"].iloc[i]),
        "description": clean_nan(df_dico["description"].iloc[i]),
        "price": clean_nan(df_dico["prix"].iloc[i]),
        "type": "NB", 
        "updated_at": df_dico["publishedAt"].iloc[i],
    }
    payload1 = {
        "name": clean_nan(df_dico["enseigneInfosvendeur"].iloc[i]),
    }
    payload2 = {
        "location": clean_nan(df_dico["location"].iloc[i]),
        "surface": clean_nan(df_dico["surf"].iloc[i]),
        "agencement": clean_nan(df_dico["agencement"].iloc[i]),
        "general": clean_nan(df_dico["general"].iloc[i]),
        "dependance": clean_nan(df_dico["dependance"].iloc[i]),
        "annexe": clean_nan(df_dico["annexe"].iloc[i]),
    }

    announcement_payloads.append(payload)
    agency_payload.append(payload1)
    caracteristic_payload.append(payload2)

In [6]:
if __name__ == "__main__":
    s = SessionLocal()
    for i in range(len(df_dico)):
        ann = create_announcement(
            s,
            announcement_data=announcement_payloads[i],
            announcer_data=agency_payload[i],
            estate_data=caracteristic_payload[i]
        )
    print("Created announcement id:", ann.id)

2025-11-26 16:58:54,885 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-11-26 16:58:54,900 INFO sqlalchemy.engine.Engine SELECT announcer.id, announcer.name, announcer.address, announcer.slogan, announcer.description, announcer.telephone, announcer.`yearMember` 
FROM announcer 
WHERE announcer.name = %(name_1)s
2025-11-26 16:58:54,902 INFO sqlalchemy.engine.Engine [generated in 0.00210s] {'name_1': 'EFFICITY Sukhanakova Elena'}
2025-11-26 16:58:54,908 INFO sqlalchemy.engine.Engine INSERT INTO announcer (name, address, slogan, description, telephone, `yearMember`) VALUES (%(name)s, %(address)s, %(slogan)s, %(description)s, %(telephone)s, %(yearMember)s)
2025-11-26 16:58:54,909 INFO sqlalchemy.engine.Engine [generated in 0.00130s] {'name': 'EFFICITY Sukhanakova Elena', 'address': None, 'slogan': None, 'description': None, 'telephone': None, 'yearMember': None}
2025-11-26 16:58:54,916 INFO sqlalchemy.engine.Engine SELECT announcements.id, announcements.ref, announcements.title, announ